In [1]:
import sys
sys.path.append('../')

In [2]:
# %load_ext autoreload
# %autoreload 2

# RAG

In [3]:
import os
import torch
import ollama
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [4]:
from src.config import PATH_PROJECT, PROMPT_SYSTEM

# PATH_PROJECT = r'C:\Users\FX506\Desktop\CS\3A\MVA\LLM\mva-llm-project'
PATH_PROJECT = r'C:\Users\cyrie\Documents\Cours\2024-2025_3A\ENS_MVA\LLM\mva-llm-project'
CHROMA_PERSIST_DIR = os.path.join(PATH_PROJECT, "data/chroma_db/")

if not os.path.exists(CHROMA_PERSIST_DIR) or not os.listdir(CHROMA_PERSIST_DIR):
    print("🆕 Création de la base Chroma et indexation des documents...")

    # Charger les fichiers PDF
    data_dir = os.path.join(PATH_PROJECT, "data/")
    documents = []

    for file in os.listdir(data_dir):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(data_dir, file)
            pdf_loader = PyMuPDFLoader(pdf_path)
            documents.extend(pdf_loader.load())

    # Diviser les documents en chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device}, encode_kwargs={'device': device})

    # Initialiser Chroma avec les embeddings 
    vectorstore = Chroma.from_documents(docs, embedding_model, persist_directory=CHROMA_PERSIST_DIR)
    vectorstore.persist()
    print("Base Chroma sauvegardée avec succès")

else:
    print("Chargement de la base Chroma existante...")
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device}, encode_kwargs={'device': device})
    vectorstore = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embedding_model)

print("Base Chroma prête à être utilisée")

Chargement de la base Chroma existante...


c:\Users\cyrie\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\cyrie\AppData\Local\Temp\ipykernel_26440\3523803818.py:36: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embedding_model)


Base Chroma prête à être utilisée


In [5]:
query = "What are the symptoms of depression ?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

for doc in retrieved_docs:
    print(doc.page_content)  # Afficher les documents les plus pertinents

259
Mood disorders
•	 Depressive symptoms include depressed mood or markedly diminished interest or 
pleasure in activities.
•	 There are multiple anxiety symptoms, which may include feeling nervous, anxious or on 
edge, not being able to control worrying thoughts, fear that something awful will happen, 
having trouble relaxing, muscle tension or sympathetic autonomic symptoms.
•	 Neither the depressive nor the anxiety symptoms – considered separately – are sufficiently
guilt, anger, denial, blame, difficulty accepting the death, feeling one has lost a part of one’s self, 
inability to experience positive mood, emotional numbness, difficulty in engaging with social or 
other activities). Some common symptoms of prolonged grief disorder are similar to those observed 
in a depressive episode (e.g. sadness, loss of interest in activities, social withdrawal, feelings of guilt,
•	 The symptoms include:
•	 at least one affective symptom such as mood lability, irritability, depressed mood or 

In [6]:
# import ollama

# context = "\n\n".join([doc.page_content for doc in retrieved_docs])
# prompt = f"Contexte:\n{context}\n\nQuestion: {query}\nRéponse:"


# response = ollama.chat(model="deepseek-coder-v2", messages=[{"role": "user", "content": prompt}])
# print(response['message'].content)

In [7]:
import pandas as pd 
df = pd.read_excel(os.path.join(PATH_PROJECT, "data/") + "dataset.xlsx").fillna("Non renseigné")

## Pandas

In [8]:
from src.inference import get_code_with_rag
from src.config import PROMPT_SYSTEM_pandas, PROMPT_SYSTEM_EXPERT_pandas
code = get_code_with_rag(model = "mistral", model_expert = "mistral",
                  w_model_expert = True,
                  prompt_system=PROMPT_SYSTEM_pandas, 
                  prompt_system_expert=PROMPT_SYSTEM_EXPERT_pandas, 
                  lan_logic = 'pandas', vectorstore=vectorstore) 
print(code)

c:\Users\cyrie\Documents\Cours\2024-2025_3A\ENS_MVA\LLM\mva-llm-project\notebooks\..\src\inference.py:222: SyntaxWarning: invalid escape sequence '\c'
  • Relevant condition names for History relation: [Condition names]'''


None


In [ ]:
from src.inference import get_code_with_rag
from src.config import PROMPT_SYSTEM_pandas, PROMPT_SYSTEM_EXPERT_pandas
get_code_with_rag(model = "mistral", model_expert = "deepseek-coder-v2",
                  w_model_expert = True,
                  prompt_system=PROMPT_SYSTEM_pandas, 
                  prompt_system_expert=PROMPT_SYSTEM_EXPERT_pandas, 
                  lan_logic = 'pandas') 

ResponseError: model "llama-3.2" not found, try pulling it first (status code: 404)

In [ ]:
from src.inference import get_code_with_rag
from src.config import PROMPT_SYSTEM_pandas, PROMPT_SYSTEM_EXPERT_pandas
get_code_with_rag(model = "deepseek-coder-v2", model_expert = "deepseek-coder-v2",
                  w_model_expert = True,
                  prompt_system=PROMPT_SYSTEM_pandas, 
                  prompt_system_expert=PROMPT_SYSTEM_EXPERT_pandas, 
                  lan_logic = 'pandas') 

## Kanren

In [13]:
from src.inference import get_code_with_rag
from src.config import PROMPT_SYSTEM_kanren ,PROMPT_SYSTEM_EXPERT_kanren 
get_code_with_rag(model = "llama3.2", model_expert = "llama3.2",w_model_expert = True,prompt_system=PROMPT_SYSTEM_kanren, prompt_system_expert=PROMPT_SYSTEM_EXPERT_kanren , lan_logic = 'karen') 

Chargement de la base Chroma existante...
Base Chroma prête à être utilisée


In [ ]:
from src.inference import get_code_with_rag
from src.config import PROMPT_SYSTEM_kanren ,PROMPT_SYSTEM_EXPERT_kanren 
get_code_with_rag(model = "deepseek-r1", model_expert = "deepseek-r1",w_model_expert = True,prompt_system=PROMPT_SYSTEM_kanren, prompt_system_expert=PROMPT_SYSTEM_EXPERT_kanren , lan_logic = 'karen') 

ResponseError: model "llama-3.2" not found, try pulling it first (status code: 404)

In [ ]:
import pandas as pd
from kanren import facts, var, run
from diagnosis_kanren_rules import Observed, History, Diagnosis, PATIENT, rules

# Charger le dataset
df = pd.read_excel('/mnt/data/dataset.xlsx').fillna("Non renseigné")

def diagnose_patient(patient_id):
    # Injecter les règles de diagnostic
    rules()

    # Extraire les données du patient
    patient_data = df[df['PatientID'] == patient_id]

    # Injecter les facts Observed
    for symptom, week in zip(patient_data["Observed_Symptom"], patient_data["Observed_Week"]):
        facts(Observed, (patient_id, symptom, week))

    # Injecter les facts History
    for condition, count in zip(patient_data["History_Condition"], patient_data["History_Count"]):
        if condition != "Non renseigné":
            facts(History, (patient_id, condition, count))

    # Exécuter la requête kanren
    x = var()
    result = run(0, x, Diagnosis(patient_id, x))
    print(f"Patient {patient_id} diagnosis: {result}")

# Exemple d'exécution pour tous les patients
for pid in df['PatientID'].unique():
    diagnose_patient(pid)


CalledProcessError: Command '['python', 'patient_1_diagnosis.py']' returned non-zero exit status 1.

In [ ]:
response = ollama.chat(
  model="llama.3.2",
  messages=[
      {"role": "system", "content": PROMPT_SYSTEM},
      {"role": "user", "content": "Please translate Bipolar I and II criteria into python pyDatalog."}
  ]
)

print(response['message'].content)

In [7]:
from src import code_Datalog 

+ code_Datalog.Observed('John', 'Mania', 4.0)

# Utiliser aussi les mêmes termes déclarés dans le même namespace
print(code_Datalog.ManicEpisode(code_Datalog.PATIENT))


PATIENT
-------
John   


In [7]:
import pyDatalog

from pyDatalog import pyDatalog

pyDatalog.clear()
pyDatalog.create_terms('Patient, p, w, Count, Observed, ManicEpisode, DepressionPeriod, BipolarI, BipolarII, PsychoticFeatures, at_least_one_depression, at_least_two_episodes, Diagnosis')

# ManicEpisode rules
ManicEpisode(p) <= Observed(p, 'Mania', w) & (w >= 3.0)
ManicEpisode(p) <= Observed(p, 'Hypomania', w) & (w >= 2.5)

# Depression rules
DepressionPeriod(p) <= Observed(p, 'MajorDepressiveDisorder', w) & ~(ManicEpisode(p))

# PsychoticFeatures rule (commented out as in your original Soufflé)
# PsychoticFeatures(p) <= Observed(p, 'Psychosis', w)

# Bipolar I diagnosis
BipolarI(p) <= ManicEpisode(p) & at_least_one_depression(p) & at_least_two_episodes(p)

# Bipolar II diagnosis (no psychosis)
BipolarII(p) <= ManicEpisode(p) & ~(PsychoticFeatures(p)) & at_least_one_depression(p) & at_least_two_episodes(p)

# Helper predicates
at_least_one_depression(p) <= DepressionPeriod(p)
# Here, pyDatalog does not have direct "count()" like Soufflé, so you'll handle counts differently (see note below)
# For now, we keep a placeholder
at_least_two_episodes(p) <= (Count >= 2)  # Will need manual count handling with Python

# Diagnosis outputs (print later)
Diagnosis(p, 'Bipolar I') <= BipolarI(p)
Diagnosis(p, 'Bipolar II') <= BipolarII(p)

+ Observed('John', 'Mania', 4.0)
+ Observed('John', 'MajorDepressiveDisorder', 3.0)
+ Observed('John', 'MajorDepressiveDisorder', 3.0)

print(ManicEpisode(p))
print(DepressionPeriod(p))
print(Diagnosis(p, X))



[]
[]


NameError: name 'X' is not defined

# LLM only approach

In [17]:
from src.inference import get_answer

In [18]:
get_answer(patient_id=10, model='mistral')

ResponseError: model "mistral" not found, try pulling it first (status code: 404)

In [21]:
get_answer(patient_id=1, model='deepseek-r1:8b')

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


: 

: 

In [ ]:
for i in range(2, 11):
    print(f"patient {i}", get_answer(patient_id=i, model='deepseek-r1:8b'))